
# Día 1 — OpenCV: Ecosistema, `cv::Mat`/NumPy, E/S y visualización

**Objetivo del día:** entender el ecosistema de OpenCV en Python, dominar la estructura de una imagen como `numpy.ndarray` (análoga a `cv::Mat`), practicar la E/S (imágenes y vídeo) y visualizar correctamente en BGR/RGB/GRAY. Además, asentaremos bases matemáticas mínimas: tensores y color lineal vs. no lineal.

> Este cuaderno forma parte de un Jupyter Book. Todos los fragmentos de código están pensados para ejecutarse en orden.



## Índice
1. Ecosistema y conceptos clave
2. `cv::Mat` vs `numpy.ndarray`: dtype, shape, strides, memoria
3. Canales de color: BGR/RGB/GRAY y conversión
4. ROI, copia vs. vista (copy vs. view)
5. E/S de imágenes: `imread`, `imwrite`
6. Vídeo con `VideoCapture`
7. Cronometría: copias vs. vistas
8. **Stretch**: `safe_imread` robusto
9. Ejercicios (con soluciones ocultables)
10. Lecturas y referencias



## 1. Ecosistema y conceptos clave

- **OpenCV** (cv2) es una librería de visión por computador con _bindings_ para Python.
- En Python, una imagen es un `numpy.ndarray` (filas × columnas × canales), análogo al `cv::Mat` de C++.
- **Tipos de datos (`dtype`)**: típicamente `uint8` (0–255), aunque también se usan `float32` / `float64` para operaciones numéricas.
- **Canales**: BGR (convención OpenCV), RGB (convención Matplotlib/PIL), y GRAY (monocanal).
- **Layout de memoria**: _row-major_ (C-order). Las **strides** indican cuántos bytes hay que saltar para pasar al siguiente elemento de cada eje.
- **ROI** (Region of Interest): submatrices que referencian la memoria original (vistas) o copias explícitas.
- **I/O**: `cv2.imread`, `cv2.imwrite`, `cv2.VideoCapture`.



## 2. Matemáticas mínimas: tensores y color lineal vs. no lineal

- Una imagen en escala de grises es un **tensor orden 2** (matriz) \(I \in \mathbb{R}^{H\times W}\). Una imagen color BGR/RGB es un **tensor orden 3** \(I \in \mathbb{R}^{H\times W\times 3}\).
- Operaciones elementales (suma, resta, producto escalar) se aplican **por elemento** (broadcasting de NumPy).
- **Espacio de color lineal vs. no lineal**: la mayoría de imágenes 8‑bit sRGB están **codificadas con una curva gamma** (no lineal). Para cálculos físicos (mezclas, filtros de energía) conviene **linealizar**:  
  \[L = V^{\gamma}\quad (\text{aprox. } \gamma \approx 2.2),\quad V=\text{valor sRGB normalizado}\]
  y al finalizar **re-aplicar** la gamma inversa para visualizar.


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Ajustes de visualización en notebook
plt.rcParams['figure.figsize'] = (8, 6)

print(cv2.__version__)


In [ ]:

# Generamos una imagen sintética de prueba (gradiente + círculo) y la guardamos
synthetic = np.zeros((240, 320, 3), dtype=np.uint8)
for y in range(synthetic.shape[0]):
    synthetic[y, :, 1] = np.uint8(255 * y / synthetic.shape[0])  # canal G con gradiente vertical
cv2.circle(synthetic, center=(160,120), radius=60, color=(255, 0, 0), thickness=-1)  # BGR: azul lleno

out_dir = Path('/mnt/data')
out_dir.mkdir(parents=True, exist_ok=True)
synthetic_path = out_dir / 'synthetic.jpg'
cv2.imwrite(str(synthetic_path), synthetic)
synthetic_path



## 3. E/S de imágenes: `imread`, `imwrite` y comprobaciones básicas


In [ ]:

# RUTA DE ENTRADA: cambia 'synthetic_path' por tu imagen si lo deseas
img_path = str(synthetic_path)  # por defecto usamos la imagen sintética
img = cv2.imread(img_path, cv2.IMREAD_COLOR)

if img is None:
    raise FileNotFoundError(f"No se pudo leer la imagen en {img_path}. Comprueba la ruta y permisos.")

print('dtype:', img.dtype)
print('shape (H, W, C):', img.shape)
print('strides (bytes):', img.strides)  # NumPy muestra strides en bytes

# Guardamos una copia
saved_path = str(out_dir / 'copy_saved.png')
cv2.imwrite(saved_path, img)
print('Guardada copia en:', saved_path)



## 4. Visualización: BGR vs. RGB en la **misma celda**
OpenCV usa **BGR**, mientras que Matplotlib espera **RGB**. Debemos convertir antes de mostrar.


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(10,4))

# Izquierda: mostrar el array en BGR (incorrecto en Matplotlib, pero lo forzamos para ilustrar la diferencia)
axes[0].imshow(img)  # interpretado como RGB => colores "raros" porque el array está en BGR
axes[0].set_title('Interpretado como RGB (input BGR)')
axes[0].axis('off')

# Derecha: conversión BGR->RGB correcta
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
axes[1].imshow(img_rgb)
axes[1].set_title('Convertido a RGB (correcto)')
axes[1].axis('off')

plt.tight_layout()
plt.show()



### Conversión a GRAY y verificación de canales


In [ ]:

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print('GRAY dtype:', gray.dtype, 'shape:', gray.shape)  # H, W
plt.imshow(gray, cmap='gray'); plt.title('GRAY'); plt.axis('off'); plt.show()



## 5. ROI, **vista** vs. **copia**
Una **ROI** con slicing NumPy suele ser **vista** (comparte memoria). Si modificas la vista, se modifica el original. Usa `.copy()` si quieres aislarla.


In [ ]:

h, w = img.shape[:2]
roi = img[h//4: 3*h//4, w//4: 3*w//4]       # ROI central (vista)
roi_copy = roi.copy()                       # copia independiente

# Modificamos la vista: dibujamos un rectángulo rojo (BGR: (0,0,255)) en la ROI
cv2.rectangle(roi, (5,5), (roi.shape[1]-6, roi.shape[0]-6), (0,0,255), 2)

fig, axes = plt.subplots(1, 3, figsize=(12,4))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)); axes[0].set_title('Imagen (modificada vía ROI)'); axes[0].axis('off')
axes[1].imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)); axes[1].set_title('ROI (vista)'); axes[1].axis('off')
axes[2].imshow(cv2.cvtColor(roi_copy, cv2.COLOR_BGR2RGB)); axes[2].set_title('ROI copy (aislada)'); axes[2].axis('off')
plt.tight_layout(); plt.show()



## 6. Strides: cómo se recorre la memoria
Las **strides** son el número de **bytes** que hay que saltar para avanzar una unidad en cada eje. En una imagen `H×W×C` `uint8`, típicamente:
- `stride_H = W*C` bytes para saltar a la siguiente fila,
- `stride_W = C` bytes para pasar al siguiente pixel en la fila,
- `stride_C = 1` byte para pasar al siguiente canal del mismo pixel.


In [ ]:

print("Strides img:", img.strides, " (bytes)")
print("dtype itemsize:", img.dtype.itemsize, "byte(s)")
print("Comprobación W*C:", img.shape[1]*img.shape[2], "vs stride fila en bytes:", img.strides[0])



## 7. Vídeo con `cv2.VideoCapture`
Abrimos un archivo o webcam. En entornos sin cámara, este bloque simplemente informará y continuará.


In [ ]:

# Intenta abrir webcam; si falla, informa. En tu entorno, cambia por un archivo de vídeo si lo deseas.
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se pudo abrir la cámara. Prueba con un archivo de vídeo en su lugar.")
else:
    ret, frame = cap.read()
    cap.release()
    if ret:
        print("Frame leído:", frame.shape, frame.dtype)
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)); plt.title('Primer frame'); plt.axis('off'); plt.show()
    else:
        print("No se pudo leer un frame de la cámara.")



## 8. Cronometría: copia vs. vista
Medimos tiempo de **copiar** (`.copy()`) frente a obtener una **vista** (slicing).


In [ ]:

import timeit

setup = "import numpy as np; import cv2; img=np.random.randint(0,256,(1080,1920,3),dtype=np.uint8)"
time_copy = timeit.timeit("roi=img[100:900,200:1700].copy()", setup=setup, number=50)
time_view = timeit.timeit("roi=img[100:900,200:1700]", setup=setup, number=50)
print(f"Copia: {time_copy:.6f} s (50 iters)")
print(f"Vista: {time_view:.6f} s (50 iters)")



## 9. **Stretch**: `safe_imread` (cargas con validación y errores con causa)
Implementamos una función robusta para leer imágenes que:
- Valida la ruta y la existencia del archivo,
- Intenta leer en modo color/gray según parámetro,
- Devuelve `(img, None)` si todo va bien; o `(None, error_message)` si algo falla.


In [ ]:

from typing import Optional, Tuple, Literal

def safe_imread(path: str, mode: Literal['color','gray']='color') -> Tuple[Optional[np.ndarray], Optional[str]]:
    """Lee una imagen con validación de ruta y errores con causa.
    
    Parameters
    ----------
    path : str
        Ruta al archivo de imagen.
    mode : {'color','gray'}
        Modo de lectura. 'color' => BGR, 'gray' => escala de grises.
    
    Returns
    -------
    (img, err) : Tuple[Optional[np.ndarray], Optional[str]]
        img: np.ndarray si ok; None si falla.
        err: None si ok; string con causa si falla.
    """
    if not isinstance(path, (str, Path)):
        return None, "El parámetro 'path' debe ser str o Path."
    p = Path(path)
    if not p.exists():
        return None, f"El archivo no existe: {p}"
    if not p.is_file():
        return None, f"La ruta no es un archivo: {p}"
    
    flag = cv2.IMREAD_COLOR if mode == 'color' else cv2.IMREAD_GRAYSCALE
    img = cv2.imread(str(p), flag)
    if img is None:
        return None, "cv2.imread devolvió None. Puede ser un formato no soportado o un archivo corrupto."
    return img, None

# Prueba con nuestra imagen sintética
ok_img, err = safe_imread(synthetic_path, mode='color')
print("OK:", ok_img is not None, "| Error:", err)



## 10. Ejercicios (con soluciones ocultables)

**Ejercicio 1.** Carga una imagen desde disco, imprime su `dtype`, `shape` y `strides`.  
**Ejercicio 2.** Muestra la misma imagen en BGR y en RGB en una **misma salida** con subplots.  
**Ejercicio 3.** Extrae una ROI central como **vista**, dibuja un rectángulo y verifica que se modifica la imagen original. Repite con `.copy()` y verifica que **no** afecta al original.  
**Ejercicio 4.** Convierte tu imagen a GRAY y guarda el resultado en PNG.  
**Ejercicio 5.** Mide el tiempo (con `timeit`) de crear una vista vs. una copia para una ROI grande.

<details>
<summary><b>Solución (mostrar/ocultar)</b></summary>

```python
# Ejercicio 1
img2, err = safe_imread(synthetic_path)
assert err is None
print(img2.dtype, img2.shape, img2.strides)

# Ejercicio 2
fig, axes = plt.subplots(1,2, figsize=(10,4))
axes[0].imshow(img2)  # BGR mal interpretado como RGB
axes[0].set_title('Interpretado como RGB (input BGR)'); axes[0].axis('off')
axes[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
axes[1].set_title('Convertido a RGB'); axes[1].axis('off')
plt.tight_layout(); plt.show()

# Ejercicio 3
h, w = img2.shape[:2]
roi_v = img2[h//4:3*h//4, w//4:3*w//4]  # vista
cv2.rectangle(roi_v, (5,5), (roi_v.shape[1]-6, roi_v.shape[0]-6), (0,255,0), 2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)); plt.title('Original modificado'); plt.axis('off'); plt.show()

img2b = img2.copy()
roi_c = img2b[h//4:3*h//4, w//4:3*w//4].copy()  # copia
cv2.circle(roi_c, (roi_c.shape[1]//2, roi_c.shape[0]//2), 20, (255,0,255), 2)
# No afecta a img2b
fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].imshow(cv2.cvtColor(img2b, cv2.COLOR_BGR2RGB)); ax[0].set_title('Original (intacto)'); ax[0].axis('off')
ax[1].imshow(cv2.cvtColor(roi_c, cv2.COLOR_BGR2RGB)); ax[1].set_title('ROI copia (modificada)'); ax[1].axis('off')
plt.tight_layout(); plt.show()

# Ejercicio 4
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
out_gray = str(out_dir / 'resultado_gray.png')
cv2.imwrite(out_gray, gray2)

# Ejercicio 5
import timeit
setup = "import numpy as np; img=np.random.randint(0,256,(1080,1920,3),dtype=np.uint8)"
print('copy:', timeit.timeit("roi=img[100:900,200:1700].copy()", setup=setup, number=50))
print('view:', timeit.timeit("roi=img[100:900,200:1700]", setup=setup, number=50))
```
</details>



## 11. Lecturas y referencias

- Documentación oficial de OpenCV (Python): https://docs.opencv.org/
- Tutoriales (OpenCV-Python Tutorials): https://docs.opencv.org/master/d6/d00/tutorial_py_root.html
- NumPy: https://numpy.org/doc/stable/
- sRGB vs. linear (curva gamma): especificación sRGB IEC 61966-2-1; explicación práctica: https://entropymine.com/imageworsener/gamma/
- Matplotlib (para visualización): https://matplotlib.org/stable/



## Anexo: `imshow(title, img, bgr=True)`
Una utilidad habitual para evitar errores de conversión de color.


In [ ]:

def imshow(title, img, bgr=True):
    plt.figure()
    if img.ndim == 2:
        plt.imshow(img, cmap='gray')
    else:
        if bgr:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
    plt.title(title)
    plt.axis('off')

# Ejemplo en una MISMA celda: mostrar BGR y RGB lado a lado usando la función
fig, axes = plt.subplots(1, 2, figsize=(10,4))
axes[0].imshow(img)  # BGR mal interpretado como RGB
axes[0].set_title('BGR interpretado como RGB'); axes[0].axis('off')
axes[1].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[1].set_title('RGB correcto'); axes[1].axis('off')
plt.tight_layout(); plt.show()
